In [1]:
import sys
sys.path.append("../")
import rg
from rg.graph import incidence_matrix
from rg.graph import ring_diagram ,show_spanning_trees, star,arc, debug_graph, tabulate_graphs,simple_ring_graph
from rg.graph import circuits, set_circuit_flow,set_source_sink_flow,apply_flow
from rg.graph import scalar_propagator
from rg.graph import save_graph_collection, load_graph_collection
#for inline dev
import numpy as np
from sympy import *

A = incidence_matrix(edges= [ [0,1],  [1,0], [-1,0], [1,-1], [1,-1], [0,-1] ], species_vector=[0,1,0,0,0,0])
debug_graph(A)

#this shows a backflow which we should not really have in the graph - good example

In [2]:
BWS = [
    incidence_matrix(edges=[[-1,0],[0,-1]], species_vector=[0,0]), #prop
    incidence_matrix(edges=[[-1,0],[0,-1]], species_vector=[1,1]), #prob B
    incidence_matrix(edges=[[-1,0],[0,-1]], species_vector=[0,1]), #trans    
    incidence_matrix(edges=[[-1,0],[0,-1],[0,-1]], species_vector=[0,0,0]), #single particle branching
    incidence_matrix(edges=[[-1,0],[0,-1],[0,-1]], species_vector=[0,0,1]), #quasi particle branching
    incidence_matrix(edges=[[-1,0],[-1,0],[0,-1],[0,-1]], species_vector=[0,1,0,1]), #single particle branching
    incidence_matrix(edges=[[-1,0],[-1,0],[0,-1]], species_vector=[1,0,1]), #single particle branching
    incidence_matrix(edges=[[-1,0],[-1,0],[0,-1],[0,-1]], species_vector=[1,0,1,1]), #single particle branching
    incidence_matrix(edges=[[-1,0],[0,-1],[0,-1],[0,-1]], species_vector=[1,1,0,1]), #single particle branching
    incidence_matrix(edges=[[-1,0],[-1,0],[0,-1],[0,-1],[0,-1]], species_vector=[0,1,0,1,1]), #single particle branching
    #incidence_matrix(edges=[[-1,0],[0,-1],[0,-1],[0,-1]], species_vector=[0,0,1,1]), #single particle branching
]

    
tabulate_graphs(BWS, options={"size": (300,150), "hide_vinfty": True})

## Generate combos of primitives

In [ ]:
import itertools
idx = list(range(len(BWS)))
mulfodder= [tup for tup in list(itertools.permutations(idx,4))]
len(mulfodder)

In [ ]:
uniqueones = {}
comb = [6,1,5,3]
comb = mulfodder[200]
print(comb)
A,B,C,D =tuple(BWS[i] for i in comb)

In [ ]:
# for i, c in enumerate(mulfodder):
#     A,B,C,D =tuple(BWS[i] for i in c)
#     for r in brute_mul(A,B,C,D):
#         uniqueones[r.graph_hash()] = r
#     if i % 50 == 0: print(i)

In [ ]:
#problem with loops hash and possible drawing colours which is a related problem because it requires understanding the distinct loop
#function colour edges from incidence matrix which can be used to construct the residual and the loop part
#then we can draw just those parts
#compute the symmetry factor of the graph
#plot the symmetry factor in the center
#graph saving function
#RISKS - The graph hash might not be right, performance is terrible, closure using shuffle product? (I think we are ok on most of these but lets see if anything is missing)
#smaller drawing with numbers on the corners

In [3]:

#save_graph_collection(sv, "test.out")
col = load_graph_collection("test.out",  True)
col[0]

incidence_matrix([[-1, -1,  1,  0,  0,  1],
                  [ 0,  0, -1,  0,  1,  0],
                  [ 0,  0,  0,  1, -1, -1],
                  [ 1,  1,  0, -1,  0,  0]])

In [4]:
simple_ring_graph(col[0])

In [5]:
simple_ring_graph(col[0])

In [6]:
lp = col[0].loop_part
simple_ring_graph(lp)

In [7]:
res = col[0].residue_part
np.where(res.sum(axis=1) != 0)[0]
debug_graph(res)

In [12]:
convergent_diags = [c for c in col if c.num_ex_edges < 5]
len(convergent_diags)

In [14]:
#MAKE convergent diagrams e.g. those that have less than 5 external legs which is a property on incidence matrix
loops = {}
#for c in col:
for c in convergent_diags:   
    lh = c.loop_hash()
    if lh not in loops: loops[lh] = { "graph" : c.loop_part, "counter" : 0 }
    loops[lh]["counter"] += 1
keys = list(loops.keys())    

In [ ]:
# import pandas as pd
# pd.DataFrame(list(loops.keys()), columns=["key"]).sort_values("key")

In [11]:
#tabulate_graphs([loops[k]["graph"] for k in keys], diagram_class=simple_ring_graph)


In [ ]:
#tabulate_graphs(col, diagram_class=simple_ring_graph)
tabulate_graphs(convergent_diags, diagram_class=simple_ring_graph)

In [ ]:
tabulate_graphs(ress,diagram_class=simple_ring_graph)

In [ ]:
# import itertools 
# def graph_admissable(g, L):
#     loops = g.first_betti_number
#     if loops > L:return False
#     #allow brudges only at tree level
#     if loops > 0 and g.bridge_count  > 1: return False
#     return True

# def graph_exists(l,g):
#     for _g in l:
#         if _g.__mul_hash__() == g.__mul_hash__(): return True
#     return False
    
# def update_set(G=[], newguys = [], L=1): 
#     mulfodder= [tup for tup in list(itertools.permutations(G,2))]
#     for m in mulfodder:
#         for s in incidence_matrix.shuffle(*m):
#             if s != (0,0): #the null case     
#                 g = m[0].dprod(m[1],species_chord_set=s)
#                 if not graph_admissable(g,L=L):continue
#                 if not graph_exists(newguys,g):newguys.append(g)
#     return newguys

# L = 1
# primitives = BWS[2:]    
# newguys = []
# newguys=update_set(primitives,newguys)

# for p in primitives:
#     for q in newguys:
#         for s in incidence_matrix.shuffle(p,q):
#             if s != (0,0): #the null case     
#                 g = p.dprod(q,species_chord_set=s)
#                 if not graph_admissable(g,L=L):continue
#                 if not graph_exists(newguys,g):
#                     newguys.append(g)
#                     print(len(newguys))


# TODO

For BWS make sure we can produce all diagrams

## Loop hash which does not care about residuals

## Other option
We can create all products of all *combinations* of 4 vertices which we know to suffice due to dimensional analysis - because 

In [ ]:
#create all 4-sequences and do a fold-left reduction 

## Draw the loops

In [ ]:
new_guys_with_loops = [n for n in newguys if n.first_betti_number > 0]
tabulate_graphs(new_guys_with_loops, diagram_class=simple_ring_graph, options={"size": (300,150), "hide_vinfty": True}, max_split=5)

In [ ]:
#create a grid for the number of vertices, bridges and loop in each object
import pandas as pd
pd.DataFrame([[g.bridge_count, g.first_betti_number, g._num_internal_edges, g.shape[0]-1, g.shape[-1]-g._num_internal_edges] for g in newguys], 
             columns=["bridges", "loops", "internal_edges", "vertices", "ex_edges"]).sort_values("loops", ascending=False)


## BWS primitives and maybe some other examples

In [ ]:
branching = incidence_matrix(edges=[[-1,0],[0,-1],[0,-1]])
s = debug_graph(branching)
s

In [ ]:
coag = incidence_matrix(edges=[[-1,0],[-1,0],[0,-1]], species_vector=[0,1,0])
debug_graph(coag)

In [ ]:
coag.species_vector

In [ ]:
shuf = incidence_matrix.shuffle(coag,branching)
res = coag.dprod(branching,species_chord_set=shuf[-1])
debug_graph(res)

In [ ]:
shuf = incidence_matrix.shuffle(branching,coag)
res = branching.dprod(coag,species_chord_set=shuf[-1])
debug_graph(res)

In [ ]:
res

In [ ]:
shuf = incidence_matrix.shuffle(coag,res)
print(shuf[-1])
res2 = coag.dprod(res,species_chord_set=shuf[-1])
debug_graph(res2)

In [ ]:
#i thin the trick is to check new 1 particular ir graphs (happly add them all but...)
#if none of the new 1P1 graphs (which excludes new tree levels) are within L then terminate
#any internal edge that is not in a circuit is a bridge i.e any edge that is in all spanning trees

#Get the generators and create vertices - maybe add something
#Have a look at them in a table - todo, better drawing
#for each pair, tabulate the shuffle product in round 1 - keep track of the new ones that are within : rule is it must add a new residual not just a new stick end
#for each new one, tabulate shuffle product with all existing - repeat until no new ones
#the question of loop topology hash... 

In [ ]:
sunset = incidence_matrix(edges=[ [0,1], [1,0], [0,1], [0,-1], [-1,1] ])
set_source_sink_flow(sunset)
ring_diagram(sunset)

In [ ]:
A = incidence_matrix(edges= [ [0,-1],  [0,-1], [-1,0], [-1,0]] ,species_vector=[4,4,4,3])
ring_diagram(A, options={"add_rotation":-30})

In [ ]:
C = incidence_matrix(edges= [ [0,-1],  [0,-1], [-1,0]] )
ring_diagram(A, options={"add_rotation":0})

In [ ]:
ring_diagram(A.dprod(C))

In [ ]:
ring_diagram(A.dprod(A))

In [ ]:
B = incidence_matrix(edges= [ [-1,0],  [-1,0], [0,1], [-1,1], [1,-1],[1,-1], [1,2], [2,-1]] )

In [ ]:
#should use this when there are no loops
star(B, options={"extra_rotation":0})

In [ ]:
A.residual(exiting=False)

In [ ]:
G=_prod_(A, A)
G      

In [ ]:

ring_diagram(G)

In [ ]:

            
RG = residual_matrix(G)
RG#ring_diagram(RG, options={"show_ex_edges"})

In [ ]:
ring_diagram(RG, options={ "add_rotation":-30})

In [ ]:
#tonight rendering of vertex diagrams (loopless ring diagrams) and residual diagrams. 
#pyramids might be nice but maybe another day

# Colourings
On a one loop graph with one species, there are as many colourings as there paths of any length between pairs of external vertices
 - which is to say, as many colourings as there are external vertices
 - if something can be expressed as a three colouring, we can always find a single pole to produce something on the propagator basis
 - adding species, even we project a large space of colourings, the graph can only every be expressed as a 3-colouring - not true, can change colour internally
 - however the kinematics from the species colouring enters trivially in the integration i think although we should check the multiple residues

In [ ]:
2**14-1

In [ ]:
A = incidence_matrix(edges= [ [0,1],  [1,0]], external_vertices=[0,1])
apply_flow(A,set_source_sink_flow)
ring_diagram(A)


In [ ]:
latex(Matrix(np.array([[1,-1,-1],[-1,1,1]])) * diag( Symbol("q_0"),Symbol("q_1"), Symbol("q3")))

In [ ]:
latex(Matrix(np.array([ [1,-1,0],[1,-1,0], [0,1,-1],[-1,0,1],[-1,0,1]]).T) * diag( Symbol("p_0"),Symbol("p_1"),Symbol("q_0"),Symbol("q_1"),Symbol("q_2")))

In [ ]:
A = incidence_matrix(edges= [ [0,1], [0,1], [1,0]],
                       external_vertices=[0,1])
apply_flow(A,set_source_sink_flow)
ring_diagram(A, options={"show_labels"})


In [ ]:
2**6-1

In [ ]:
show_spanning_trees(A, show_labels=True)

In [ ]:
latex(A.kirchhoff)

In [ ]:
latex(Matrix(A[:2,:3]))

In [ ]:
latex(Matrix(A.edge_label_matrix))


In [ ]:
L = A.edge_label_matrix * A.edge_label_matrix.T
L

In [ ]:
for s in A._internal_symbols_(): L=L.subs({s**2:s})
        #treat all external edges as z so we can get a polynomial in z for forest order
for s in A._external_symbols_(): L=L.subs({s**2:Symbol("z")}) 
L

In [ ]:
red =  incidence_matrix.__sym_i_cofactor__(L) 
latex(Matrix(red))


In [ ]:
latex(Matrix(L))

In [ ]:
B = incidence_matrix(3,3,
                       [ [0,1], [1,2], [2,0]],
                       external_vertices=[0,1])
apply_flow(B,set_source_sink_flow)
ring_diagram(B)

In [ ]:
B.constraint_system(True)

In [ ]:
C = incidence_matrix(3,3,
                       [ [0,1], [1,2], [2,0]],
                       external_vertices=[0,1,2])
apply_flow(C,set_source_sink_flow)
ring_diagram(C)

In [ ]:
C.constraint_system(True)

In [ ]:
D = incidence_matrix(4,4,
                       [ [0,1], [1,2], [2,3], [3,0]],
                       external_vertices=[0,1,2])
apply_flow(D,set_source_sink_flow)
ring_diagram(D, )#options={"show_labels"} messed up label

In [ ]:
#3 linearly independent internal edges can be generated from (3,1)
D.constraint_system(True)

In [ ]:
E = incidence_matrix(5,5,
                       [ [0,1], [1,2], [2,3], [3,4], [4,0]],
                       external_vertices=[0,1,2])
apply_flow(E,set_source_sink_flow)
ring_diagram(E, )#options={"show_labels"} messed up label

In [ ]:
#3 linearly independent internal edges can be generated from (4,1)
E.constraint_system(True)

In [ ]:
F = incidence_matrix(5,5,
                       [ [0,1], [1,2], [2,3], [3,4], [4,0]],
                       external_vertices=[0,1,4])
apply_flow(F,set_source_sink_flow)
ring_diagram(F, )#options={"show_labels"} messed up label

In [ ]:
#2 linearly independant? - no actually , just a trick p4 is on its own where as above, p1 is on its own
#actually 3 - so there are as many as there paths i.e. as many as there are externals
F.constraint_system(True)

In [ ]:
G = incidence_matrix(5,5,
                       [ [0,1], [1,2], [2,3], [3,4], [4,0]],
                       external_vertices=[0,2,4])
apply_flow(G,set_source_sink_flow)
ring_diagram(G, )#options={"show_labels"} messed up label

In [ ]:
G.constraint_system(True)

In [ ]:
#The graph will have islands of internal nodes which will have the same momenta. 
#So no matter how many external nodes there are, if they are contiguous we minimise the number of colourings

## work in progress

What is the best method for determining the edge basis. Cycles can be used. 
- Important to partition cycles into positive and negative frequencies
- Import to attribute to each edge the correct edge basis

### from the constraints at vertices, we try to construct an edge basis 
 - need to be careful about Sympy interface because I am not sure about some things yet

In [ ]:
sunset = incidence_matrix(3,2,
                       [ [0,1], [1,0], [0,1]  ],
                       external_vertices=[0,1])

L = sunset.first_betti_number
maps = sunset.edges_to_loop_basis()
#these are the ids of each edge in the incidence matrix and their coefficients on the edge basis
#if there are two loops and two externals each edge takes a vector: [L1|L2|EX1|EX2]
maps

In [ ]:
#given an edge basis and a known loop number, create the propagator for each edge - here we know to use the first L as the loop basis
scalar_propagator(maps[0][:L], maps[0][L:])

In [ ]:
#apply_flow(sunset, set_source_sink_flow)
#sunset.flip_edges(0)
ring_diagram(sunset)

### all the propagators look like this

In [ ]:
rg._product_([scalar_propagator(maps[i][:L], maps[i][L:]).value for i in maps.keys()])

In [ ]:
#below is displayed with the source sink flow used to solve constraints and not the loop flow...
#but we can see from the propagators that e0 and e1 flow in the *some* direction and e1 flows opposite for each loop variable
#it is important that the loop variables are represented with opposite signs for the set of propagaors
ring_diagram(sunset, options={"show_labels"})

In [ ]:
apply_flow(sunset,set_circuit_flow)
ring_diagram(sunset, options={"show_labels"})

### example of the shorthand 
- convention must be known but useful to create a larger integral from matrix of coefficients

In [ ]:
mat = [[0,1,0,1,1,1],
       [0,-1,0,0,2,2]]
scalar_propagator.from_basis_vector(mat,2)

# A first look at the incidence matrix class

In [ ]:
itest = incidence_matrix.__sample__()
itest

In [ ]:
INC = incidence_matrix.__sample__()
#INC.flip_edges(1)
D = ring_diagram(INC, options={ "show_labels":True, }) 
D

In [ ]:
all_circuits=  circuits(INC,sort=True)
all_circuits

# now take a look at one of the spanning rees

In [ ]:
#show_spanning_trees(INC)

In [ ]:
tree = INC.spanning_trees.iloc[-1].values
tree_cuts = INC.edge_complement(tree)
print(tree_cuts)
C = ring_diagram(INC, options={"show_labels" :True,  "cut_edges": tree_cuts}) 
C#._repr_html_()

# Edge Orientations
given cicuits, we can actually rank them by closeness to root and then we can incrementally orientate them in different ways

(1) Choose a source and sink delegate on each circuit and have allow flow towards the sink

(2) Respect loop momentum

In [ ]:
#todo the last circuit is degenerate
INC = incidence_matrix.__sample__()
all_circuits = circuits(INC,INC.spanning_trees.iloc[0].values)
print(all_circuits)

# set default circuit flow

In [ ]:
INC = incidence_matrix.__sample__()
#todo technicall need to reload cicuts but they are ok
all_circuits = circuits(INC,INC.spanning_trees.iloc[0].values)
circ = all_circuits[3]
#print(circ)
res = set_circuit_flow(INC,circ)
#print(res)
#print("edge comp", circ)
ring_diagram(INC, options={"show_labels" :True,  "cut_edges": INC.edge_complement(circ)}) 

In [ ]:
INC.spanning_trees.iloc[0].values

# circuit source sink flow

In [ ]:
INC = incidence_matrix.__sample__()
#todo technicall need to reload cicuts but they are ok
all_circuits = circuits(INC,INC.spanning_trees.iloc[0].values)
circ = all_circuits[1]
#print(circ)
res = set_source_sink_flow(INC,circ)
print("source,sink",res)
#print(res)
#print("edge comp", circ)
ring_diagram(INC, options={"show_labels" :True,  "cut_edges": INC.edge_complement(circ)}) 

# all together now...

cicuits are ordered based on the average depth of vertices in the cicuit. Outer rings score best

We process all cicuits in that order

Suppose we dont remember anything from the last processed circuit...

In [ ]:
apply_flow(INC,set_source_sink_flow)
ring_diagram(INC, options={"show_labels" :True}) 

# Other examples

In [ ]:
simp = incidence_matrix(2,2,
                       [ [0,1], [1,0]  ],
                       external_vertices=[0,1])

sunset = incidence_matrix(3,2,
                       [ [0,1], [1,0], [0,1]  ],
                       external_vertices=[0,1])

bog = incidence_matrix(7,6,
                       [ [0,1], [1,3], [0,2], [3,2], [3,5], [2,4], [4,5]  ],
                       external_vertices=[0,1,4,5])

#get cicuits using any spanning tree and produce a sink flow ordering
set_source_sink_flow(simp,circuits(simp)[0])
#simp.constraint_system(True)
ring_diagram(simp)

In [ ]:
show_spanning_trees(simp)

In [ ]:
simp = incidence_matrix(3,3,
                       [ [0,1], [1,2], [2,0]  ],
                       external_vertices=[0,1])
ring_diagram(simp)

In [ ]:
show_spanning_trees(simp)

In [ ]:
simp = incidence_matrix(4,4,
                       [ [0,1], [1,2], [2,3], [3,0]  ],
                       external_vertices=[0,1])
ring_diagram(simp)

In [ ]:
INC.kirchhoff

In [ ]:
INC = incidence_matrix.__sample__()
show_spanning_trees(INC,is_directed=False)
#len(INC.spanning_trees)

In [ ]:
#TODO: visual style for and implementation of multiple external legs
#perhaps an option for simple diagrams to force certain types of loops and ignore hierarchy

#save svg todo find save html tp pdf solution
#with open ("test.svg", "w") as f:
#    f.write(ring_diagram(sunset, options={"show_labels"})._repr_html_())